In [357]:
# Importing libraries

import sqlite3
import pandas as pd
import numpy as np
import json

## Step 1: Extract

### 1 - Customer Profiles from SQLite database: company_data.db

In [358]:
# Create SQLite database
conn = sqlite3.connect('company_data.db')
cursor = conn.cursor()

In [359]:
# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = cursor.fetchall()
print("Tables in database:", tables)

Tables in database: [('Customers',), ('unified_data',)]


In [360]:
# Save customers table into data frame
df_customers = pd.read_sql_query("SELECT * FROM customers", conn)
df_customers

,CustId,FirstName,LastName,Age,Country
0,45000,Chathura,Rathnayake,24,Sri Lanka
1,45001,Zoe,Lee,37,Australia
2,45002,Léa,Martin,22,France
3,45003,David,Anderson,36,USA
4,45004,Jessica,Miller,26,USA
...,...,...,...,...,...
115,45115,Naledi,Van Wyk,32,South Africa
116,45116,Mia,Brown,26,Australia
117,45117,Emily,Jones,28,USA
118,45118,Chloé,Petit,28,France


Query on the company_data.db (Optional - but recommended to get more pracice)

In [361]:
pd.read_sql_query("""
                  SELECT FirstName, Age FROM customers
                  WHERE Age>39""", conn)

,FirstName,Age
0,Hugo,40
1,Leon,40
2,Leon,40
3,Mia,40


### 2 - Read csv file: "transactions.csv"

In [362]:
df_transactions = pd.read_csv("transactions.csv")
df_transactions

,customer_id,transaction_id,product,amount,date
0,45000,1,Headphones,788,2025-01-01
1,45001,2,Headphones,76,2025-01-02
2,45002,3,Tablet,209,2025-01-03
3,45003,4,Headphones,277,2025-01-04
4,45004,5,Laptop,996,2025-01-05
...,...,...,...,...,...
295,45058,296,Laptop,490,2025-10-23
296,45004,297,Camera,224,2025-10-24
297,45097,298,Tablet,816,2025-10-25
298,45059,299,Phone,651,2025-10-26


### 3 - Read JSON file: "feedback.json"

In [363]:
# Read JSON file into DataFrame
with open('feedback.json') as f:
    data = json.load(f)
df_feedback = pd.DataFrame(data)

print("JSON Feedback Sample:")
print(df_feedback.head(), "\n")

JSON Feedback Sample:
   custID  rating           feedback
0   45000       1    Good experience
1   45001       3  Excellent service
2   45002       5  Delivery was late
3   45003       2  Excellent service
4   45004       2            Average 



## Step 2: Transform

In [364]:
# Clean and align column names
df_customers.rename(columns={'CustId': 'customer_id'}, inplace=True)
df_customers

df_feedback.rename(columns={'custID': 'customer_id'}, inplace=True)
df_feedback

,customer_id,rating,feedback
0,45000,1,Good experience
1,45001,3,Excellent service
2,45002,5,Delivery was late
3,45003,2,Excellent service
4,45004,2,Average
...,...,...,...
116,45116,5,Average
117,45117,5,Good experience
118,45118,4,Product quality issue
119,45119,4,Excellent service


In [365]:
df_customers

,customer_id,FirstName,LastName,Age,Country
0,45000,Chathura,Rathnayake,24,Sri Lanka
1,45001,Zoe,Lee,37,Australia
2,45002,Léa,Martin,22,France
3,45003,David,Anderson,36,USA
4,45004,Jessica,Miller,26,USA
...,...,...,...,...,...
115,45115,Naledi,Van Wyk,32,South Africa
116,45116,Mia,Brown,26,Australia
117,45117,Emily,Jones,28,USA
118,45118,Chloé,Petit,28,France


In [366]:
# Merge all sources
df_merged = (
    df_customers
    .merge(df_transactions, on='customer_id', how='left')
    .merge(df_feedback, on='customer_id', how='left')
)

In [367]:
# Clean and handle missing values
df_merged['feedback'] = df_merged['feedback'].fillna('No feedback')
df_merged['rating'] = df_merged['rating'].fillna(0).astype(int)

In [368]:
df_merged.isna().sum()

customer_id       0
FirstName         0
LastName          0
Age               0
Country           0
transaction_id    0
product           0
amount            0
date              0
rating            0
feedback          0
dtype: int64

## Step 3: Load

In [369]:
#  Load Unified Data into SQLite
df_merged.to_sql('unified_data', conn, if_exists='replace', index=False)

df_merged

,customer_id,FirstName,LastName,Age,Country,transaction_id,product,amount,date,rating,feedback
0,45000,Chathura,Rathnayake,24,Sri Lanka,1,Headphones,788,2025-01-01,1,Good experience
1,45000,Chathura,Rathnayake,24,Sri Lanka,158,Camera,1112,2025-06-07,1,Good experience
2,45001,Zoe,Lee,37,Australia,2,Headphones,76,2025-01-02,3,Excellent service
3,45001,Zoe,Lee,37,Australia,139,Camera,983,2025-05-19,3,Excellent service
4,45001,Zoe,Lee,37,Australia,246,Laptop,968,2025-09-03,3,Excellent service
...,...,...,...,...,...,...,...,...,...,...,...
295,45117,Emily,Jones,28,USA,285,Phone,1126,2025-10-12,5,Good experience
296,45118,Chloé,Petit,28,France,119,Laptop,688,2025-04-29,4,Product quality issue
297,45118,Chloé,Petit,28,France,148,Tablet,325,2025-05-28,4,Product quality issue
298,45118,Chloé,Petit,28,France,275,Camera,804,2025-10-02,4,Product quality issue


Query on the unified dataset

In [370]:
query1 = """
SELECT FirstName, LastName, product, date, feedback
FROM unified_data
WHERE country= 'Sri Lanka'
"""
print("Sri Lankan customers:")
pd.read_sql_query(query1, conn)

Sri Lankan customers:


,FirstName,LastName,product,date,feedback
0,Chathura,Rathnayake,Headphones,2025-01-01,Good experience
1,Chathura,Rathnayake,Camera,2025-06-07,Good experience
2,Rashmi,Perera,Tablet,2025-01-28,Product quality issue
3,Kasun,Perera,Camera,2025-02-25,Average
4,Kasun,Perera,Camera,2025-06-17,Average
5,Isuru,Wickramasinghe,Tablet,2025-03-03,Excellent service
6,Isuru,Wickramasinghe,Tablet,2025-09-06,Excellent service
7,Isuru,Wickramasinghe,Camera,2025-09-20,Excellent service
8,Isuru,Wickramasinghe,Phone,2025-09-23,Excellent service
9,Kavindi,Perera,Phone,2025-03-07,Good experience


In [371]:
df_merged['feedback'].value_counts()

feedback
Delivery was late        71
Product quality issue    69
Average                  62
Good experience          58
Excellent service        40
Name: count, dtype: int64

In [372]:
query2 = """
SELECT product, country
FROM unified_data
where feedback= 'Delivery was late'
"""
print("Late deliveries")
df_late = pd.read_sql_query(query2, conn)

print("By product:")
print(df_late['product'].value_counts())

print("\nBy country:")
print(df_late['Country'].value_counts())

Late deliveries
By product:
product
Tablet        18
Headphones    17
Laptop        13
Phone         12
Camera        11
Name: count, dtype: int64

By country:
Country
Germany         15
Italy           12
Sri Lanka       11
USA             11
France           7
South Africa     5
Australia        5
England          5
Name: count, dtype: int64


In [373]:
query3 = """
SELECT country, COUNT(DISTINCT customer_id) as num_customers,
       ROUND(AVG(amount),2) as avg_spent
FROM unified_data
GROUP BY country
ORDER BY avg_spent DESC
"""
print("Average Spending by Country:")
print(pd.read_sql_query(query3, conn))

Average Spending by Country:
        Country  num_customers  avg_spent
0           USA             20     704.27
1  South Africa             11     663.97
2       England             15     655.30
3         Italy             17     650.92
4        France             17     647.66
5     Sri Lanka             11     636.37
6     Australia             16     617.58
7       Germany             13     599.86


In [374]:
query4 = """
SELECT rating, COUNT(*) as num_feedbacks,
       ROUND(AVG(amount),2) as avg_amount
FROM unified_data
GROUP BY rating
ORDER BY rating DESC
"""
print("Customer Ratings vs Average Transaction Amount:")
print(pd.read_sql_query(query4, conn), "\n")

Customer Ratings vs Average Transaction Amount:
   rating  num_feedbacks  avg_amount
0       5             73      608.40
1       4             47      633.49
2       3             52      655.60
3       2             65      623.95
4       1             63      736.46 



### Exporting integrated (unified) dataset

In [375]:
# Export Integrated Dataset
df_merged.to_csv('unified_dataset.csv', index=False)
print("Integrated dataset exported to unified_dataset.csv")

Integrated dataset exported to unified_dataset.csv


In [376]:
# Close the connection
conn.close()